In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive


In [0]:
!mkdir yet_another_skip_gram

In [0]:
%cd yet_another_skip_gram

/content/gdrive/My Drive/yet_another_skip_gram


In [0]:
!wget https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip

--2019-12-24 18:01:16--  https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip
Resolving wit3.fbk.eu (wit3.fbk.eu)... 217.77.80.8
Connecting to wit3.fbk.eu (wit3.fbk.eu)|217.77.80.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [plain/text]
Saving to: ‘get.php?path=XML_releases%2Fxml%2Fted_en-20160408.zip’

get.php?path=XML_re     [      <=>           ]  15.29M  14.1MB/s    in 1.1s    

2019-12-24 18:01:17 (14.1 MB/s) - ‘get.php?path=XML_releases%2Fxml%2Fted_en-20160408.zip’ saved [16035163]



In [0]:
!unzip get.php?path=XML_releases%2Fxml%2Fted_en-20160408.zip

Archive:  get.php?path=XML_releases%2Fxml%2Fted_en-20160408.zip
  inflating: ted_en-20160408.xml     


In [0]:
# -*- encoding:utf-8 -*-
# !bin/env python
import lxml.etree as e
import re
import string
from collections import defaultdict
import numpy as np
import time
from tqdm import tqdm

doc = e.parse('./ted_en-20160408.xml')
input_text = '\n'.join(doc.xpath('//content/text()'))
MAX_EXP = 6
EXP_TABLE_SIZE = 1000
EXP_TABLE = []
for i in range(0, EXP_TABLE_SIZE+1):
    EXP_TABLE.append(np.exp(((i-EXP_TABLE_SIZE/2)/(EXP_TABLE_SIZE/2)) * MAX_EXP))

def tokenize(corpus):
    # Pre-Processing Data
    ## remove parenthesis 
    clean_text = re.sub(r"\([^)]*\)", "", corpus)
    ## tokenlize
    sentences = []
    for paragraph in clean_text.split('\n'):
        if paragraph:
            for sentence in paragraph.split('.'):
                if sentence:
                    clean_sent = re.sub(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]+\ *", " ", sentence)
                    tokens = clean_sent.lower().split()
                    sentences.append(tokens)
    return sentences


class Word2Vec(object):
    
    def __init__(self, sentences, wv_size=100, window=5, min_count=5, sample=1e-4, negative=15, alpha=0.36, min_alpha=0.0001, sg=1):
        np.random.seed(1)
        self.sentences = sentences
        self.wv_size = wv_size
        self.window = window
        self.min_count = min_count
        self.sample = sample
        self.negative = negative
        self.alpha = alpha
        self.min_alpha = min_alpha
        self.sg = sg
        self.vocab = self.vocab()
        self.input_embedding = np.random.uniform(low=-0.5/(wv_size**(3/4)), high=0.5/(wv_size**(3/4)), size=(len(self.vocab), wv_size))
        self.output_weights = np.zeros([len(self.vocab), wv_size])
        self.word_oh_v = np.zeros(len(self.vocab))
        self.G0 = np.zeros_like(self.input_embedding)
        self.G1 = np.zeros_like(self.output_weights)
        self.fudge_factor = 1e-6
    
    def vocab(self, ):
        # sentences: list of sentence token lists
        # [['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], [], ...]
        sentences = self.sentences
        vocab = defaultdict(dict)
        vocab_words = ['int']
        vocab['int']['word_count'] = 0 
        vocab_size = 0
        for sent_tokens in sentences:
            vocab_size += len(sent_tokens)
            for word in sent_tokens:
                if not word.isdigit() and word not in vocab:
                    vocab[word]['word_count'] = 1
                    vocab_words.append(word)
                else:
                    if word.isdigit():
                        vocab['int']['word_count'] += 1 
                    else:
                        vocab[word]['word_count'] += 1
        low_freq_words = []
        for word in vocab:
            if vocab[word]['word_count'] < self.min_count:
                low_freq_words.append(word)
        for word in low_freq_words:
            vocab_size -= vocab[word]['word_count']
            del vocab[word]
            vocab_words.remove(word)
        sorted_vocab = []
        for word in vocab:
            sorted_vocab.append((word, vocab[word]['word_count']))
        sorted_vocab.sort(key=lambda tup: tup[1], reverse=True)
        for idx, word in enumerate(sorted_vocab):
            vocab[word[0]]['word_freq'] = vocab[word[0]]['word_count'] / vocab_size
            vocab[word[0]]['word_index'] = idx
        return vocab
       
    # Forward Propagation
    def train_batch_sg(self, ):
        sentences = self.sentences
        vocab = self.vocab
        train_step = 0
        neg_word_list = self.neg_sampling(vocab)
        for sent_tokens in tqdm(sentences):
            clean_sent = []
            for word in sent_tokens:
                if word.isdigit():
                    word = 'int'
                if word not in vocab:
                    continue
                # Subsampling of High-Freq Word
                keep_prob = min((np.sqrt(vocab[word]['word_freq'] / self.sample) + 1) * (self.sample / vocab[word]['word_freq']), 1)
                keep_list = [1] * int(keep_prob * 1000) + [0] * (1000 - int(keep_prob * 1000))
                if keep_list[np.random.randint(1000)]:
                    clean_sent.append(word)
            for pos, center_word in enumerate(clean_sent):
                b = np.random.randint(0, self.window)
                for pos_c, context in enumerate(clean_sent[max(0, pos - (self.window - b)) : pos + (self.window - b) + 1], max(0, pos - (self.window - b))):
                    if pos_c != pos:
                        train_step += 1
                        # Adaptive 
                        context_idx = self.vocab[context]['word_index']
                        if np.min(self.G0) != 0 and self.alpha/np.min(self.G0) < self.min_alpha:
                            print(train_step)
                        self.train_pair_sg(center_word, context, neg_word_list=neg_word_list, neg=self.negative)

        # Save the final embedding vector matrix
        fname1 = './word_embedding_vector_matrix_test_%sf1.txt' % str(train_step)
        np.savetxt(fname1, self.input_embedding)
        fname2 = './word_embedding_vector_matrix_test_%sf2.txt' % str(train_step)
        np.savetxt(fname2, (self.input_embedding + self.output_weights) / 2)
                
    # Back Propagation
    def train_pair_sg(self, center_w, context_w, neg_word_list, neg=0):
        if neg > 0:
            context_idx = self.vocab[context_w]['word_index']
            center_idx = self.vocab[center_w]['word_index']
            neg_sample = [(center_w, 1)]
            wv_h = self.input_embedding[context_idx]
            # wv_j = self.input_embedding[self.vocab[context_w]['word_index']]
            for i in range(0, neg):
                neg_word = neg_word_list[np.random.randint(0, len(neg_word_list))] 
                if (neg_word, 0) not in neg_sample and neg_word != center_w:
                    neg_sample.append((neg_word, 0))
            # log(P(Wo|Wi)) = log(sigmoid(np.dot(Vt, Vi))) + np.sum(sigmoid(-np.dot(Vn, Vi))  for neg_w in neg_sample[1:]) / (len(neg_sample) - 1)
            # Adagrad
            dh = np.zeros(self.wv_size)
            for neg_w in neg_sample:
                target, label = neg_w[0], neg_w[1]
                neg_w_idx = self.vocab[target]['word_index']
                wv_j = self.output_weights[neg_w_idx]
                dwjh = self.sigmoid(np.dot(wv_h, wv_j)) - label
                dwj = dwjh * wv_h
                self.G1[neg_w_idx] += np.power(dwj, 2)
                dwj /= np.sqrt(self.G1[neg_w_idx]) + self.fudge_factor
                assert dwj.shape == wv_j.shape
                dh += dwjh * wv_j
                # Update the output weight matrix
                self.output_weights[neg_w_idx] -= self.alpha * dwj
            # Update the input embedding matrix
            self.G0[context_idx] += np.power(dh, 2)
            dh /= np.sqrt(self.G0[context_idx]) + self.fudge_factor
            assert dh.shape == wv_h.shape
            self.input_embedding[context_idx] -= self.alpha * dh
    
    # Negative Sampling
    def neg_sampling(self, vocab):
        NEG_SIZE = 1e6
        neg_word_list = []
        sorted_vocab = []
        freq_sum = np.sum(vocab[word]['word_freq']**0.75 for word in vocab)
        for word in vocab:
            sorted_vocab.append((word, vocab[word]['word_freq']))
        sorted_vocab.sort(key=lambda tup: tup[1], reverse=True)
        for word in sorted_vocab:
            neg_word_list.extend([word[0]] * int((word[1]**0.75 / freq_sum) * NEG_SIZE))
        return neg_word_list
    
    def sigmoid(self, x):
        if x > MAX_EXP:
            x = MAX_EXP
        if x < -MAX_EXP:
            x = -MAX_EXP
        exp_x = EXP_TABLE[int((-x / MAX_EXP) * 500 + 500)]
        return 1 / (1 + exp_x)
                        
    def cosine_distance(self, vec1, vec2):
        assert vec1.shape == vec2.shape
        return np.dot(vec1, vec2) / (np.sqrt(np.dot(vec1, vec1)) * np.sqrt(np.dot(vec2, vec2)))
    
    def most_similar(self, word, topn):
        most_similar_word = []
        word_idx = self.vocab[word]['word_index']
        for w in self.vocab:
            w_idx = self.vocab[w]['word_index']
            if w_idx != word_idx:
                cos_dis = self.cosine_distance(self.input_embedding[word_idx], self.input_embedding[w_idx])
                most_similar_word.append((w, cos_dis))
        most_similar_word.sort(key=lambda tup : tup[1], reverse=True)
        return most_similar_word[0:topn]
    
    
if __name__ == '__main__':
    sentences = tokenize(input_text)
    word2vec = Word2Vec(sentences)
    vocabulary = word2vec.vocab
    b1 = time.time()
    word2vec.train_batch_sg()
    train_time = time.time() - b1
    most_similar_word_science = word2vec.most_similar('science', topn=10)
    most_similar_word_man = word2vec.most_similar('man', topn=10)
    most_similar_word_kill = word2vec.most_similar('kill', topn=10)
    most_similar_word_red = word2vec.most_similar('red', topn=10)
    most_similar_word_monday = word2vec.most_similar('monday', topn=10)

In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

In [16]:
word2vec.input_embedding

(21347, 100)

In [0]:
!ls -l

In [0]:
!head word_embedding_vector_matrix_test_1819403f1.txt

-3.282024875107556611e-02 -5.826200858811838613e-03 -2.942360690395902090e-01 1.034315980324577861e-01 -4.963040083117526910e-03 -2.606238945256724726e-01 4.704008686119513782e-02 -6.182750940313269834e-03 -3.124411382949281168e-02 4.648520097759412800e-02 2.985719901746192173e-02 1.264142813446313685e-01 6.743878016947289777e-02 1.077093402805473776e-01 -7.160478417889054592e-02 -2.177848342892459849e-02 -1.073037389942793074e-01 -1.092832064078366194e-01 1.269299739254704795e-01 2.921413045564457550e-02 2.121110973501567215e-02 6.527352257496758792e-02 -6.302715658293760181e-02 5.194575884338862443e-02 -6.606398243266244086e-02 -9.308226486143702724e-03 1.237372055347956551e-02 3.727326064827547747e-02 -4.306124569442359301e-02 3.120414225391085483e-02 -1.301461865639449711e-01 1.288256942538826277e-01 -1.731509618174175702e-01 1.942458376348581428e-01 -3.772635616223784377e-02 -7.030939305653033378e-02 1.157698193029951944e-01 1.171020177167960974e-01 1.015795205133304679e-02 3.9535

In [0]:
len(vectors)

In [22]:
vec = open('vectors.txt', 'w')
vec.write(str(21336)+' 100\n')
for indx, vector in enumerate(word2vec.input_embedding.tolist()):
  line = list(word2vec.vocab.keys())[indx]+' '
  for i in vector:
    line+=str(i)+' '
  if indx == (21336-1):
    print('last')
    vec.write(line[:-1])
  else:
    vec.write(line[:-1]+'\n')

last


In [19]:
!wget https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt

--2019-12-24 20:10:24--  https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.02s   

2019-12-24 20:10:24 (27.6 MB/s) - ‘questions-words.txt’ saved [603955/603955]



In [23]:
!head vectors.txt

21336 100
int -0.003315984542946178 0.14787348348850035 0.07458287842349483 0.162049619327259 -0.1516086864708609 -0.2927179025088564 0.05256967623858606 -0.023864974512526827 0.04613268704021846 -0.10920109222385715 -0.09714544752695269 -0.0675624039573703 0.03433204686069198 -0.11774851885741291 0.05614485060062586 0.0552582851085693 -0.12461415533819448 -0.07389037594310656 0.13168174967716365 0.039320976374574976 0.10441218247582643 0.224999667030373 -0.5881431380552276 0.11691560906900399 -0.06392365096106942 0.029610827365910152 -0.24977586677520378 0.06016950696901742 0.0594421377087568 -0.013356124669030174 0.0905526309707139 -0.1882270492902598 -0.3188642101728276 -0.06160447009594583 0.10967212821448691 0.18383594641023593 0.06132537312201545 0.0028371216107462697 0.012506071998079325 -0.10856722682728569 0.1274421890197709 -0.015179735562567732 -0.08190274948775735 0.5538835973950628 0.07159670096583776 -0.1173937441184596 0.3046641390008516 -0.10036998699570636 0.0585124448

In [24]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [25]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



def print_accuracy(model, questions_file):
    print('Evaluating...\n')
    acc = model.accuracy(questions_file)

    sem_correct = sum((len(acc[i]['correct']) for i in range(5)))
    sem_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5))
    sem_acc = 100*float(sem_correct)/sem_total
    print('\nSemantic: {:d}/{:d}, Accuracy: {:.2f}%'.format(sem_correct, sem_total, sem_acc))
    
    syn_correct = sum((len(acc[i]['correct']) for i in range(5, len(acc)-1)))
    syn_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5,len(acc)-1))
    syn_acc = 100*float(syn_correct)/syn_total
    print('Syntactic: {:d}/{:d}, Accuracy: {:.2f}%\n'.format(syn_correct, syn_total, syn_acc))
    return (sem_acc, syn_acc)

word_analogies_file = 'questions-words.txt'
accuracies = []
print('\nLoading Gensim embeddings')
print('Accuracy for Word2Vec:')
accuracies.append(print_accuracy(model, word_analogies_file))



Loading Gensim embeddings
Accuracy for Word2Vec:
Evaluating...



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.evaluate_word_analogies() instead).
  
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-12-24 20:12:25,800 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
2019-12-24 20:12:26,696 : INFO : capital-common-countries: 0.0% (0/272)
2019-12-24 20:12:28,


Semantic: 0/1694, Accuracy: 0.00%
Syntactic: 0/8415, Accuracy: 0.00%

